In [3]:
import os
import sys
import glob
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyts.image import GramianAngularField

In [4]:
if sys.platform == "win32":
    dataPath = "C:\\Users\\KOTC\\Desktop\\Test2\\result\\"
elif sys.platform == "darwin":
    dataPath = "/Users/jwh/Documents/KOTC/수중도킹/2024년도/6_자기장센서 시험/Test0730/Test2/result/"

fileNames = [file for file in sorted(glob.glob(dataPath + "*.csv")) if os.path.isfile(file)]

sensorNames = [f"s{i}" for i in np.arange(1, 7, 1)]

fileIdx = [1, 8, 9, 10, 11, 12, 13, 14, 15, 2, 3, 4, 5, 6, 7]

numRows = 1920

fileList = [fileNames[idx] for idx in fileIdx]

files = [pd.read_csv(file, sep= ",", nrows= numRows) for file in fileList]

In [5]:
base = pd.read_csv(fileNames[0], sep= ",", nrows= numRows)
meanBase = pd.DataFrame(base.mean(axis= 0), columns= ['base'])

In [6]:
def calibration(df):
    res = [df.iloc[:, i] - meanBase.iloc[i, 0] for i in range(6)]
    res = np.array(res).reshape(6, -1).T
    res = pd.DataFrame(res, columns= sensorNames)
    return res

In [7]:
calibrated = [calibration(files[i]) for i in range(len(files))]
calibrated = np.array(calibrated)

print(calibrated.shape)

(15, 1920, 6)


In [8]:
gadf = GramianAngularField(image_size= 16, method= 'difference')

def sliding_window(data, window_size= 16, step_size= 1):
    n_cases, n_samples, n_sensors= data.shape
    n_windows = (n_samples - window_size) // step_size + 1     # 슬라이딩 윈도우 개수

    gadf_images = np.zeros((n_cases, n_windows, 16, 16, n_sensors))

    for i in range(n_cases):
        for j in range(n_windows):
            # 슬라이딩 윈도우로 데이터 분ㄴ할
            window_data = data[i, j : j + window_size, :]

            # 각 센서에 대해 GADF 변환
            for k in range(n_sensors):
                sensor_image = gadf.fit_transform(window_data[:, k].reshape(1, -1))
                gadf_images[i, j, :, :, k] = sensor_image[0]
    
    return gadf_images

In [9]:
windowSize = 16
stepSize = 1

In [10]:
encodingResult = sliding_window(calibrated,
                                window_size= windowSize,
                                step_size= stepSize
                                )

print(encodingResult.shape)

(15, 1905, 16, 16, 6)


In [11]:
inIdx = [0, 1, 10]
leftIdx = [2, 7, 8]
rightIdx = [5, 11, 12]

inClass = [encodingResult[idx] for idx in inIdx]
leftClass = [encodingResult[idx] for idx in leftIdx]
rightClass = [encodingResult[idx] for idx in rightIdx]

inClass = np.array(inClass)
leftClass = np.array(leftClass)
rightClass = np.array(rightClass)

print(inClass.shape)
print(leftClass.shape)
print(rightClass.shape)

(3, 1905, 16, 16, 6)
(3, 1905, 16, 16, 6)
(3, 1905, 16, 16, 6)


In [12]:
inCounter = 0
leftCounter = 0
rightCounter = 0

inClassImg = []
leftClassImg = []
rightClassImg = []

inLabelList = []
leftLabelList = []
rightLabelList = []

yIn = []
yLeft = []
yRight = []

for i in range(3):
    for j in range(inClass.shape[1]):
        inImg1 = inClass[i, j, :, :, :3]
        inImg2 = inClass[i, j, :, :, 3 : 6]
        leftImg1 = leftClass[i, j, :, :, :3]
        leftImg2 = leftClass[i, j, :, :, 3:6]
        rightImg1 = rightClass[i, j, :, :, :3]
        rightImg2 = rightClass[i, j, :, :, 3:6]

        inClassImg.append(inImg1)
        inLabelList.append(f"I_{inCounter}")
        yIn.append(0)
        inCounter += 1

        leftClassImg.append(leftImg1)
        leftLabelList.append(f"L_{leftCounter}")
        yLeft.append(1)
        leftCounter += 1

        rightClassImg.append(rightImg1)
        rightLabelList.append(f"R_{rightCounter}")
        yRight.append(2)
        rightCounter += 1

        inClassImg.append(inImg2)
        inLabelList.append(f"I_{inCounter}")
        yIn.append(0)
        inCounter += 1

        leftClassImg.append(leftImg2)
        leftLabelList.append(f"L_{leftCounter}")
        yLeft.append(1)
        leftCounter += 1

        rightClassImg.append(rightImg2)
        rightLabelList.append(f"R_{rightCounter}")
        yRight.append(2)
        rightCounter += 1

inClassImg = np.array(inClassImg)
leftClassImg = np.array(leftClassImg)
rightClassImg = np.array(rightClassImg)

print(inClassImg.shape, leftClassImg.shape, rightClassImg.shape)

(11430, 16, 16, 3) (11430, 16, 16, 3) (11430, 16, 16, 3)


In [13]:
inLabelIdx = np.array(inLabelList)
leftLabelIdx = np.array(leftLabelList)
rightLabelIdx = np.array(rightLabelList)

print(inLabelIdx.shape, leftLabelIdx.shape, rightLabelIdx.shape)

(11430,) (11430,) (11430,)


In [14]:
yIn = np.array(yIn)
yLeft = np.array(yLeft)
yRight = np.array(yRight)

print(yIn.shape, yLeft.shape, yRight.shape)

(11430,) (11430,) (11430,)


In [15]:
magnetic = np.concatenate([inClassImg, leftClassImg, rightClassImg])
labels = np.concatenate([yIn, yLeft, yRight])
index = np.concatenate([inLabelIdx, leftLabelIdx, rightLabelIdx])

tmp = [[x, y, z] for x, y, z in zip(magnetic, labels, index)]

random.shuffle(tmp)

X_data = [n[0] for n in tmp]
y_data = [n[1] for n in tmp]
i_data = [n[2] for n in tmp]

np.savez(os.path.join("C:\\Users\\KOTC\\Desktop\\0903\\", "GADF_3Class_16Sample_SlidingWindow"), X_data= X_data, y_data= y_data, i_data= i_data)